In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
# Read in the data
overtime_hours = pd.read_csv('../data/csv/work_hours_overtime.csv', index_col='date', parse_dates=True)
base_hours = pd.read_csv('../data/csv/work_hours_base.csv', index_col='date', parse_dates=True)
portions = pd.read_csv('../data/csv/portions.csv', index_col='date', parse_dates=True)
employees = pd.read_csv('../data/csv/employees.csv', index_col='employee', dtype={'employee': str})

overtime_hours.columns.name='employee'
base_hours.columns.name='employee'

# Calculate total work hours
total_hours = overtime_hours + base_hours
total_hours.head()

employee,4000175,4000182,4000220,4000239,4000240,4000241,4000243,4000244,4000245,4000246,...,9753767,9784946,9795581,9798848,9814343,9869495,9893478,9926881,9965178,9972531
date,,,,,,,,,,,,,,,,,,,,,
2022-08-23,525,732,585,627,732,627,732,627,480,732,...,633,640,898,530,480,908,480,1257,750,605
2022-08-24,1245,585,732,732,480,732,732,627,480,732,...,604,641,495,480,532,480,604,480,601,607
2022-08-25,480,732,732,732,732,732,627,480,480,732,...,495,642,480,994,536,894,605,531,599,603
2022-08-26,525,585,732,627,732,627,480,627,480,732,...,745,492,902,480,850,598,603,528,589,603
2022-08-27,480,732,732,732,627,627,732,480,480,732,...,494,642,494,531,895,911,747,480,602,589


In [3]:
# Splitting the job titles into different categories
food_related = [
    'آشپز 1',
    'آشپز 2',
    'سر آشپز',
    'سر شیفت',
    'سر شیفت آماده سازی',
    'سر شیفت برنج',
    'سردخانه دار',
    'سرپرست تولید',
    'قصاب',
    'کمک آشپز',
    'کمک انباردار',
    'انباردار',
    'اپراتور انبار',
]

distribution_related = [
    'سرگارسون',
    'گارسون',
    'کارگر رستوران',
    'کارگر ساده',
]

hr_related = [
    'تعمیرکار',
    'متصدی اداری',
    'متصدی برنامه ریزی و سفارشات',
    'مدیرپروژه',
    'مسئول امور اداری و منابع انسانی',
    'کارشناس',
    'کارشناس سلامت ایمنی محیط',
    'کارمند اداری',
]

In [4]:
# Total daily portions
total_daily_portions = portions.sum(axis=1)

# Categorize work hours
categorized_work_hours = total_hours.transpose().groupby(employees['job_title'])
portion_related_categorized_work_hours = categorized_work_hours.sum()
total_daily_employees = categorized_work_hours.count().sum()

In [5]:
# Extract related work hours
food_work_hours = portion_related_categorized_work_hours.loc[food_related] # food related
distribution_work_hours = portion_related_categorized_work_hours.loc[distribution_related] # distribution related

# Divide the total daily work hours by the total daily portions for each job category
food_time_cost = food_work_hours.divide(total_daily_portions, axis=1).transpose()
distribution_time_cost = distribution_work_hours.divide(total_daily_portions, axis=1).transpose()

# Calculate HR related time cost (total work hours of hr related employees / total number of employees)
hr_work_hours = categorized_work_hours.sum().loc[hr_related] # hr related
hr_time_cost = hr_work_hours.divide(total_daily_employees, axis=1).transpose()

In [11]:
# Plot the time cost tables
fig = go.Figure()
fig.add_trace(go.Scatter(x=hr_time_cost.index, y=hr_time_cost.sum(axis=1), name='HR'))
fig.add_trace(go.Scatter(x=distribution_time_cost.index, y=distribution_time_cost.sum(axis=1), name='Distribution'))
fig.add_trace(go.Scatter(x=food_time_cost.index, y=food_time_cost.sum(axis=1), name='Food'))
fig.update_layout(
    title='Time Cost',
    xaxis_title='Date',
    yaxis_title='Time Cost (minutes/portion(employee for HR))',
    legend_title='Job Category',
)
fig.show()